In [1]:
from torch import nn, sqrt
import torch
import sys
from math import sqrt

import sys
sys.path.append('../')

from CoAtNet import CoAtNet

In [2]:
x=torch.randn(2,3,224,224)
coatnet=CoAtNet(3,224)
out = coatnet(x)
print(out.shape)

torch.Size([2, 1000])


In [3]:
# Data loading code
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets

path = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(path, 'train')
valdir = os.path.join(path, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

In [11]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None),
    num_workers=4, pin_memory=True, sampler=train_sampler)

In [12]:
# (train_sampler is None)

In [13]:
images, labels = next(iter(train_loader))

In [14]:
out = coatnet(images)

In [19]:
# labels

In [16]:
print(out.shape)

torch.Size([32, 1000])


In [17]:
torch.argmax(out, dim = -1)

tensor([178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178,
        178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 178,
        178, 178, 178, 178])

In [ ]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda(args.gpu)

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)